<a href="https://colab.research.google.com/github/lahirunie-dulsara/EN3150-Assignment-3-CNN/blob/Aazir/appendicities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install ucimlrepo

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
# Path to your zip file in Drive
zip_path = '/content/drive/MyDrive/US_Pictures.zip'

# Extract the zip file to a local folder in Colab
extract_dir = '/content/US_Pictures'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Dataset extracted to:", extract_dir)

Mounted at /content/drive
Dataset extracted to: /content/US_Pictures


In [4]:
import os
import re
from PIL import Image

dataset_dir = extract_dir  # path where the dataset was extracted

selected_images = []
selected_filenames = []

# Regular expression: match 'app' or 'appendix' as the only word (after ignoring leading numbers)
pattern = re.compile(r'^(?:[\d\.\s_-]*)(app|appendix)$', re.IGNORECASE)

# Allowed file extensions
allowed_exts = ('.bmp', '.png')

for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.lower().endswith(allowed_exts):
            # Remove file extension
            name_without_ext = os.path.splitext(file)[0]
            # Remove leading/trailing spaces
            cleaned_name = name_without_ext.strip()

            # Match only if the cleaned name fits the pattern
            if pattern.match(cleaned_name):
                img_path = os.path.join(root, file)
                img = Image.open(img_path).convert('L')  # Convert to grayscale
                selected_images.append(img)
                selected_filenames.append(file)

print(f"Total selected images: {len(selected_images)}")

Total selected images: 679


In [5]:
# Path to the folder where you want to save selected images
save_dir = '/content/Selected_Images'

# Create the folder if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Loop through selected images and save them
for img, filename in zip(selected_images, selected_filenames):
    save_path = os.path.join(save_dir, filename)
    img.save(save_path)  # PIL will save in the original format automatically

print(f"Saved {len(selected_images)} images to {save_dir}")

Saved 679 images to /content/Selected_Images


In [24]:
import os, re

image_dir = '/content/Selected_Images'
image_data = []

for img in selected_filenames:
    match = re.match(r"(\d+)", img)
    if match:
        us_number = int(match.group(1))
        image_data.append({"Image Name": img, "US_Number": us_number})

image_df = pd.DataFrame(image_data)
print(image_df.head())


       Image Name  US_Number
0   887.3 App.bmp        887
1  96.12 App .bmp         96
2  747.10 App.bmp        747
3   152.2 App.bmp        152
4   918.2 App.bmp        918


In [22]:
import pandas as pd

file_path = '/content/drive/MyDrive/app_data.xlsx'

df = pd.read_excel(file_path)
print(df.head())


     Age   BMI     Sex  Height  Weight  Length_of_Stay    Management  \
0  12.68  16.9  female   148.0    37.0             3.0  conservative   
1  14.10  31.9    male   147.0    69.5             2.0  conservative   
2  14.14  23.3  female   163.0    62.0             4.0  conservative   
3  16.37  20.6  female   165.0    56.0             3.0  conservative   
4  11.08  16.9  female   163.0    45.0             3.0  conservative   

        Severity Diagnosis_Presumptive        Diagnosis  ...  \
0  uncomplicated          appendicitis     appendicitis  ...   
1  uncomplicated          appendicitis  no appendicitis  ...   
2  uncomplicated          appendicitis  no appendicitis  ...   
3  uncomplicated          appendicitis  no appendicitis  ...   
4  uncomplicated          appendicitis     appendicitis  ...   

   Abscess_Location  Pathological_Lymph_Nodes Lymph_Nodes_Location  \
0               NaN                       yes                 reUB   
1               NaN                       

In [30]:
merged_df = pd.merge(image_df, df[['US_Number', 'Diagnosis']], on='US_Number', how='left')

print("\n Final Table:")
print(merged_df)



 Final Table:
         Image Name  US_Number     Diagnosis
0     887.3 App.bmp        887  appendicitis
1    96.12 App .bmp         96  appendicitis
2    747.10 App.bmp        747           NaN
3     152.2 App.bmp        152  appendicitis
4     918.2 App.bmp        918  appendicitis
..              ...        ...           ...
674   169.5 App.bmp        169  appendicitis
675   550.1 App.bmp        550  appendicitis
676   838.1 App.bmp        838           NaN
677  102.12 App.bmp        102           NaN
678   841.5 App.bmp        841           NaN

[679 rows x 3 columns]


In [31]:
# Drop rows with NaN in the Diagnosis column
cleaned_df = merged_df.dropna(subset=['Diagnosis'])

In [42]:
!rm -rf Labeled_Data/appendicitis
!rm -rf Labeled_Data/no_appendicitis

In [43]:
import shutil

# --- Step 5: Create output folders ---
base_dir = '/content/Labeled_Data'
os.makedirs(f'{base_dir}/appendicitis', exist_ok=True)
os.makedirs(f'{base_dir}/no_appendicitis', exist_ok=True)

# --- Step 6: Copy images into labeled folders ---
for _, row in cleaned_df.iterrows():
    # Get image filename
    image_file = row['Image Name']
    src = os.path.join(image_dir, image_file)

    # Check diagnosis safely
    diagnosis = str(row['Diagnosis']).lower()

    if 'no' in diagnosis:
        dst = os.path.join(base_dir, 'no_appendicitis', image_file)
    else:
        dst = os.path.join(base_dir, 'appendicitis', image_file)

    # Copy image
    shutil.copy2(src, dst)

print("\n Images sorted into appendicitis / no_appendicitis folders successfully!")



 Images sorted into appendicitis / no_appendicitis folders successfully!


In [44]:
import os

# Folder paths
appendicitis_folder = '/content/Labeled_Data/appendicitis'
no_appendicitis_folder = '/content/Labeled_Data/no_appendicitis'

# Count files in each folder
num_appendicitis = len(os.listdir(appendicitis_folder))
num_no_appendicitis = len(os.listdir(no_appendicitis_folder))

print(f"Number of images in 'appendicitis' folder: {num_appendicitis}")
print(f"Number of images in 'no_appendicitis' folder: {num_no_appendicitis}")


Number of images in 'appendicitis' folder: 493
Number of images in 'no_appendicitis' folder: 79


In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
dataset = datasets.ImageFolder(root=save_dir, transform=transform)

# Split: 70% train, 15% val, 15% test
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)


FileNotFoundError: Couldn't find any class folder in /content/Selected_Images.